In [1]:
import foolbox
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import MNIST
import torchvision.transforms as transforms

In [2]:
class MnistModel(nn.Module):
    def __init__(self):
        super(MnistModel, self).__init__()
        # mnist의 경우 28*28의 흑백이미지(input channel=1)이다.
        self.conv1 = nn.Conv2d(1, 32, kernel_size = 5, padding=2)
        # feature map의 크기는 14*14가 된다
        # 첫번재 convolution layer에서 나온 output channel이 32이므로 2번째 input도 32
        self.conv2 = nn.Conv2d(32, 64, kernel_size = 5, padding=2)
        # feature map의 크기는 7*7이 된다
        # fc -> fully connected, fc는 모든 weight를 고려해서 만들기 때문에 cnn에서는 locally connected를 이용하여 만든다.
        # nn.Linear에서는 conv를 거친 feature map을 1차원으로 전부 바꿔서 input을 한다. 이게 64*7*7
        self.fc1 = nn.Linear(64*7*7, 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, 64*7*7) # linear에 들어갈 수 있도록 reshape
        x = F.relu(self.fc1(x)) # fully connected에 relu 적용
        x = F.dropout(x, training=self.training) # 가중치 감소만으로는 overfit을 해결하기가 어려움, 그래서 뉴런의 연결을 임의로 삭제
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [3]:
def advtrain(model, device, train_loader, optimizer, epoch, log_interval, train_loader_origin):
    model.train()
    avg_loss = 0
    # in training loop:
    for batch_idx, (data, target) in enumerate(train_loader):
        # gpu나 cpu로 데이터를 옮김
        data, target = data.to(device), target.to(device)
        # gradient descent전에 gradient buffer를 0으로 초기화 하는 역할을 한다
        optimizer.zero_grad()
        output = model(data)
        # negative log-likelihood: nll loss, 딥러닝 모델의 손실함수이다
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step() # Does the update based on the current gradient stored in grad
        # 여기서 기존에 저장되어있던 gradient를 기반으로 update 하기 때문에 위의 초기화가 필요함
        avg_loss+=F.nll_loss(output, target, reduction='sum').item()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader_origin.dataset),
                100. * batch_idx / len(train_loader_origin), loss.item()))
    avg_loss/=len(train_loader_origin.dataset)
    return avg_loss

In [4]:
def test(model, device, images):
    # Accuracy counter
    correct = 0
    failed_examples = []
    correct_examples = []

    # test set의 모든 예제를 test한다
    for image, label in images:
        # cpu나 gpu로 데이터를 전송한다
        image, label = image.to(device), label.to(device)

        # Re-classify the perturbed image
        output = model(image)

        # Check for success
        final_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        if final_pred.item() == label.item():
            correct += 1
            if (len(correct_examples) < 5):
                adv_ex = image.squeeze().detach().cpu().numpy()
                correct_examples.append( (label.item(), final_pred.item(), adv_ex) )
        else:
            if len(failed_examples) < 5:
                adv_ex = image.squeeze().detach().cpu().numpy()
                failed_examples.append( (label.item(), final_pred.item(), adv_ex) )

    # final_acc = correct/idx
    final_acc = correct/float(len(images))
    print("Test Accuracy = {} / {} = {}".format(correct, len(images), final_acc))

    # Return the accuracy and an adversarial example
    return final_acc, correct_examples, failed_examples

In [5]:
train_batch_size=64
test_batch_size=1
epochs=50
lr = 0.01
momentum = 0.5
seed = 1
log_interval = 1000

In [6]:
is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')

In [7]:
pretrained_model = './model/mnist_um.pth'
model = MnistModel().to(device).eval()
model.load_state_dict(torch.load(pretrained_model, map_location=device))

<All keys matched successfully>

In [8]:
# momentum: 기울기에서 속도의 개념을 추가, 기울기 업데이트시 폭을 조절한다
# lr: learning rate
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

# mean=0.5 std=1.0으로 Normalize한다
mnist_transform = transforms.Compose([
    transforms.ToTensor(), 
])

In [9]:
download_path = './data'
train_dataset = MNIST(download_path, transform=mnist_transform, train=True, download=True)
test_dataset = MNIST(download_path, transform=mnist_transform, train=False, download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=train_batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=test_batch_size,
                                          shuffle=True)


In [10]:
def deepfool_attack(model, loader):
    print("Attack image!")
    
    perturbed_images = []
    for image, label in loader:
        attack = foolbox.attacks.DeepFoolAttack(model)
        image_np = attack(image.numpy(), label.numpy())
        perturbed_image = torch.from_numpy(image_np)
        perturbed_images.append((perturbed_image, label))
        
    print("Attack done!")
    return perturbed_images

In [11]:
fmodel = foolbox.models.PyTorchModel(model, bounds=(0, 1), num_classes=10)

In [12]:
perturbed_test = deepfool_attack(fmodel, test_loader)

Attack image!
Attack done!


In [13]:
perturbed_train = deepfool_attack(fmodel, train_loader)

Attack image!
Attack done!


In [14]:
train_losses = []
test_losses = []
accuracy_list = []

for epoch in range(1, epochs + 1):
    trn_loss = advtrain(model, device, perturbed_train, optimizer, epoch, log_interval, train_loader)
    train_losses.append(trn_loss)
    accuracy, L2_correct, L2_failed = test(model, device, perturbed_test)
    accuracy_list.append(accuracy)

Train Epoch: 1 [0/60000 (0%)]	Loss: 13.331134
Test Accuracy = 1937 / 10000 = 0.1937
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.111191
Test Accuracy = 2267 / 10000 = 0.2267
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.100425
Test Accuracy = 2334 / 10000 = 0.2334
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.065625
Test Accuracy = 2636 / 10000 = 0.2636
Train Epoch: 5 [0/60000 (0%)]	Loss: 0.026779
Test Accuracy = 2989 / 10000 = 0.2989
Train Epoch: 6 [0/60000 (0%)]	Loss: 0.012778
Test Accuracy = 2796 / 10000 = 0.2796
Train Epoch: 7 [0/60000 (0%)]	Loss: 0.011426
Test Accuracy = 2831 / 10000 = 0.2831
Train Epoch: 8 [0/60000 (0%)]	Loss: 0.010224
Test Accuracy = 3148 / 10000 = 0.3148
Train Epoch: 9 [0/60000 (0%)]	Loss: 0.022377
Test Accuracy = 2920 / 10000 = 0.292
Train Epoch: 10 [0/60000 (0%)]	Loss: 0.006654
Test Accuracy = 3035 / 10000 = 0.3035
Train Epoch: 11 [0/60000 (0%)]	Loss: 0.004472
Test Accuracy = 2895 / 10000 = 0.2895
Train Epoch: 12 [0/60000 (0%)]	Loss: 0.025095
Test Accuracy = 3013 / 10000 = 0.3013
T

In [15]:
PATH = './model/mnist_deepfool_model.pth'
torch.save(model.state_dict(), PATH)

In [16]:
def get_pf_images(model, device, perturbed_data):
    model.eval()
    test_loss = 0
    correct = 0
    failed_image = []
    correct_image = []
    
    for idx, (data, target) in enumerate(perturbed_data):
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
        # dim 1에서 가장 큰 값을 가지는 것의 index를 반환
        pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
        final_pred = output.max(1, keepdim=True)[1]
        # pred.eq(data)는 pred와 data 배열이 일치하는지를 검사
        # target을 pred처럼 보이게 만들어서 .sum()을 이용해 일치하는 것들의 개수를 구한다
        # .item()은 tensor의 값을 스칼라로 만드는 역할을 한다
        correct += pred.eq(target.view_as(pred)).sum().item()
        for j in pred.eq(target.view_as(pred)):
            if j == 1:
                correct_image.append((data, target.item(), final_pred.item()))
            else:
                failed_image.append((data, target.item(), final_pred.item()))
    return correct_image, failed_image
                

In [20]:
from torchvision.utils import save_image

correct, failed = get_pf_images(model, device, perturbed_test)
SAVE_PATH = './examples/deepfool/'

In [22]:
for idx, (image, label, pred) in enumerate(failed):
    if idx < 10:
        title = SAVE_PATH + 'fail/' + str(idx) +'_'+ str(label) + '_' + str(pred) +'.png'
        save_image(image, title)
    else: break

In [23]:
for idx, (image, label, pred) in enumerate(correct):
    if idx < 10:
        title = SAVE_PATH + 'correct/' + str(idx) +'_'+ str(label) + '_' + str(pred) +'.png'
        save_image(image, title)
    else: break